# Imports

In [1]:
import numpy as np
import torch
from qYOLO.module import QTinyYOLOv2
from qYOLO.train import train, YOLOout

# Import/Train Network

> pretrained = True<br>
img_dir = "./../../Dataset/images"<br>
lbl_dir = "./../../Dataset/labels"<br>
weight_bit_width = 8<br>
act_bit_width    = 8<br>
n_anchors        = 5<br>
n_epochs         = 10<br>
batch_size       = 10<br>

> net = QTinyYOLOv2(n_anchors, weight_bit_width, act_bit_width)<br>
if pretrained:<br>
    net_path = f"./train_out/trained_net_W{weight_bit_width}A{act_bit_width}_a{n_anchors}.pth"<br>
    net.load_state_dict(torch.load(net_path))<br>
    anchors_path = f"./anchors_W{weight_bit_width}A{act_bit_width}_a{n_anchors}.txt"<br>
    anchors = torch.zeros((n_anchors, 2))<br>
    with open(anchors_path) as f:<br>
        for n, data in enumerate(f.readlines()):<br>
            anchors[n] = torch.from_numpy(np.array(data.split(',')).astype(float))<br>
        f.close()<br>
else:<br>
    net, anchors = train(img_dir, lbl_dir, weight_bit_width=weight_bit_width, act_bit_width=act_bit_width, n_anchors=n_anchors, n_epochs=n_epochs, batch_size=batch_size)

In [4]:
batch_size, w, h, out_size, n_anchs  = 10, 16, 9, 5, 4
test = torch.rand((batch_size,h,w,out_size*n_anchs))
test = test.flatten(1,2)
print(f"shape 1: {test.shape}")
test = test.unflatten(-1,(n_anchs, out_size))
print(f"shape 1: {test.shape}")

shape 1: torch.Size([10, 144, 20])
shape 1: torch.Size([10, 144, 4, 5])


# FINN

## Imports

In [ ]:
# FINN-Brevitas imports
from brevitas.export.onnx.generic.manager import BrevitasONNXManager

# ONNX libraries
import onnx
import onnx.numpy_helper as nph
import onnxruntime as rt

# Network display methods - Netron
from finn.util.visualization import showInNetron

# FINN Network Preperation imports
from finn.core.modelwrapper import ModelWrapper
from qonnx.util.cleanup import cleanup_model
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from finn.transformation.general import GiveUniqueNodeNames
from finn.util.pytorch import ToTensor
from finn.transformation.merge_onnx_models import MergeONNXModels
from finn.core.datatype import DataType
from finn.transformation.insert_topk import InsertTopK
from finn.transformation.streamline import Streamline
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.general import RemoveUnusedTensors
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition
from finn.custom_op.registry import getCustomOp

## Brevitas Export

In [ ]:
onnx_export_path = f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/og_net.onnx"
BrevitasONNXManager.export(net, (1, 3, 360, 640), onnx_export_path)

## Networks Preperation

### Original

In [ ]:
model = ModelWrapper(f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/og_net.onnx")
model = cleanup_model(model)
model = model.transform(ConvertQONNXtoFINN())

model.save(f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/tidy_net.onnx")
showInNetron(f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/tidy_net.onnx")

### Add Pre/Post-Processing

In [ ]:
model = ModelWrapper(f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/tidy_net.onnx")

# pre-processing
in_name = model.graph.input[0].name
in_shape = model.get_tensor_shape(in_name)
totensor = ToTensor()
BrevitasONNXManager.export(totensor, in_shape, f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/preproc_net.onnx")
pre_model = ModelWrapper(f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/preproc_net.onnx")
model = model.transform(MergeONNXModels(pre_model))
in_name = model.graph.input[0].name
model.set_tensor_datatype(in_name, DataType["UINT8"])

# post-processing
# TODO - check if I can actually create the output layer
model = cleanup_model(model)
model = model.transform(ConvertQONNXtoFINN())

model.save(f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/preproc_net.onnx")
showInNetron(f"./onnx_W{weight_bit_width}A{act_bit_width}_a{n_anchors}/preproc_net.onnx")

## Streamline